In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

/tmp/ipykernel_9475/2443102998.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
path_to_db_local = '/home/aleks/Рабочий стол/Ya.Phyton/6 NPSproject/telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
SELECT 
    us.user_id,
    us.lt_day,
    case when lt_day <= 365 then 'новый' else 'старый' end as is_new,
    us.age,
    us.gender_segment,
    us.os_name,
    us.cpe_type_name,
    loc.country,
    loc.city,
    SUBSTRING(age.title,4) as age_segment,
    SUBSTRING(trf.title,4) as traffic_segment,
    SUBSTRING(lft.title,4) as lifetime_segment,
    us.nps_score,
    case 
        when nps_score BETWEEN 9 and 10 then 'cторонники'
        when nps_score BETWEEN 7 and 8 then 'нейтралы'
        else 'критики'
    end as nps_group
FROM user us
left join location loc on loc.location_id = us.location_id
left join age_segment age on age.age_gr_id = us.age_gr_id
left join traffic_segment trf on trf.tr_gr_id = us.tr_gr_id
left join lifetime_segment lft on lft.lt_gr_id = us.lt_gr_id
"""

In [4]:
df = pd.read_sql(query, engine)
df.head()

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,старый,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,cторонники
1,A001WF,2344,старый,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,cторонники
2,A003Q7,467,старый,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,cторонники
3,A004TB,4190,старый,44.0,1.0,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,cторонники
4,A004XT,1163,старый,24.0,0.0,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,cторонники


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    501192 non-null  float64
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(2), int64(2), object(10)
memory usage: 53.7+ MB


In [6]:
df.to_csv('/home/aleks/Рабочий стол/Ya.Phyton/6 NPSproject/telecomm_csi_tableau.csv', index=False)